In [1]:
from langdetect import detect

def detect_language(text):
    try:
        return detect(text)
    except:
        return "unknown"


In [2]:
from pdf2image import convert_from_path

def convert_pdf_to_images(pdf_path):
    images = convert_from_path(pdf_path)
    return images


In [3]:
import cv2
import pytesseract
from pdf2image import convert_from_path

def ocr_with_language(pdf_path, lang='eng'):
    images = convert_from_path(pdf_path)
    text = ""
    for image in images:
        # Convert PIL image to OpenCV format
        open_cv_image = cv2.cvtColor(np.array(image), cv2.COLOR_RGB2BGR)
        
        # Pre-process the image
        gray = cv2.cvtColor(open_cv_image, cv2.COLOR_BGR2GRAY)
        gray = cv2.medianBlur(gray, 3)
        gray = cv2.adaptiveThreshold(gray, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY, 11, 2)
        
        # Perform OCR
        custom_config = f'--oem 3 --psm 6 -l {lang}'
        page_text = pytesseract.image_to_string(gray, config=custom_config)
        text += page_text + "\n"
    return text


Kode fra Llama

In [1]:
import cv2
import pytesseract
from pdf2image import convert_from_path
import pandas as pd

def ocr_with_language(pdf_path, lang='eng'):
    images = convert_from_path(pdf_path)
    text = ""
    layout_info = []
    for image in images:
        # Convert PIL image to OpenCV format
        open_cv_image = cv2.cvtColor(np.array(image), cv2.COLOR_RGB2BGR)
        
        # Pre-process the image
        gray = cv2.cvtColor(open_cv_image, cv2.COLOR_BGR2GRAY)
        gray = cv2.medianBlur(gray, 3)
        gray = cv2.adaptiveThreshold(gray, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY, 11, 2)
        
        # Perform OCR
        custom_config = f'--oem 3 --psm 6 -l {lang}'
        page_text = pytesseract.image_to_string(gray, config=custom_config)
        text += page_text + "\n"
        
        # Get layout information
        layout_data = pytesseract.image_to_data(gray, config=custom_config, output_type='dict')
        layout_df = pd.DataFrame(layout_data)
        layout_info.append(layout_df)
    
    return text, layout_info


In [ ]:
import cv2
import pytesseract
from pdf2image import convert_from_path
import pandas as pd
import json

def ocr_with_language(pdf_path, lang='eng'):
    images = convert_from_path(pdf_path)
    text = ""
    layout_info = []
    for image in images:
        # Convert PIL image to OpenCV format
        open_cv_image = cv2.cvtColor(np.array(image), cv2.COLOR_RGB2BGR)
        
        # Pre-process the image
        gray = cv2.cvtColor(open_cv_image, cv2.COLOR_BGR2GRAY)
        gray = cv2.medianBlur(gray, 3)
        gray = cv2.adaptiveThreshold(gray, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY, 11, 2)
        
        # Perform OCR
        custom_config = f'--oem 3 --psm 6 -l {lang}'
        page_text = pytesseract.image_to_string(gray, config=custom_config)
        text += page_text + "\n"
        
        # Get layout information
        layout_data = pytesseract.image_to_data(gray, config=custom_config, output_type='dict')
        layout_df = pd.DataFrame(layout_data)
        layout_info.append(layout_df)
    
    # Convert layout information to JSON-like string
    layout_json = []
    for layout_df in layout_info:
        layout_dict = layout_df.to_dict(orient='records')
        layout_json.append(json.dumps(layout_dict))
    
    return text, layout_json


In [4]:
import unicodedata

def normalize_text(text):
    return unicodedata.normalize('NFKD', text).encode('ASCII', 'ignore').decode('utf-8')



In [5]:
import os

In [6]:
root = "../../Scans"

In [7]:
files = os.walk(root)

In [8]:
filenames = [z for x,y,z in files]

In [9]:
filepaths = [os.path.join(root,z) for z in filenames[0]] 

In [19]:
import cv2
import pytesseract
from pdf2image import convert_from_path
from langdetect import detect
import pandas as pd
import numpy as np
import json

# Ensure you have the correct path to the Tesseract executable
#pytesseract.pytesseract.tesseract_cmd = r'C:\Program Files\Tesseract-OCR\tesseract.exe'  # Change this path if needed

# Convert PDF to images
def convert_pdf_to_images(pdf_path):
    return convert_from_path(pdf_path)

# Perform OCR with language-specific configuration
def ocr_with_language(pdf_path, lang='eng'):
    images = convert_pdf_to_images(pdf_path)
    text = ""
    for image in images:
        # Convert PIL image to OpenCV format
        open_cv_image = cv2.cvtColor(np.array(image), cv2.COLOR_RGB2BGR)
        
        # Pre-process the image
        gray = cv2.cvtColor(open_cv_image, cv2.COLOR_BGR2GRAY)
        gray = cv2.medianBlur(gray, 3)
        gray = cv2.adaptiveThreshold(gray, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY, 11, 2)
        
        # Perform OCR
        custom_config = f'--oem 3 --psm 6 -l {lang}'
        page_text = pytesseract.image_to_string(gray, config=custom_config)
        text += page_text + "\n"
    return text

def detect_and_map_language(text):
    try:
        detected_lang = detect(text)
        # Mapping to Tesseract language codes
        language_mapping = {
            'en': 'eng',
            'da': 'dan',
            'de': 'deu',
            'fr': 'fra',
            'es': 'spa',
            'it': 'ita',
            'nl': 'nld',
            'sv': 'swe',
            'no': 'nor',  # General Norwegian
            'nob': 'nor', # Norwegian Bokmål
            'nno': 'nor'  # Norwegian Nynorsk
        }
        return language_mapping.get(detected_lang, 'eng')  # Default to 'eng' if language not found
    except:
        return 'eng'

# Example list of PDF file paths and languages (for testing purposes)
file_paths = filepaths
languages = ['eng', 'deu','fra','nor']  # Example detected languages

results = []

for file_path in file_paths:
    text = ocr_with_language(file_path, lang='eng')
    detected_lang = detect_and_map_language(text)
    if detected_lang != 'eng':
        text = ocr_with_language(file_path, lang=detected_lang)
    results.append({
        'file': file_path,
        'language': detected_lang,
        'text': text
    })

# Convert results to a DataFrame
df_results = pd.DataFrame(results)
print(df_results)

# Convert the DataFrame to a dictionary
results_dict = df_results.to_dict(orient='records')

# Save to JSON for further processing using the json library
with open('ocr_results.json', 'w', encoding='utf-8') as json_file:
    json.dump(results_dict, json_file, ensure_ascii=False, indent=4)


                           file language  \
0    ../../Scans/Doc3894OCR.pdf      nor   
1    ../../Scans/Doc3893OCR.pdf      dan   
2    ../../Scans/Doc3892OCR.pdf      eng   
3    ../../Scans/Doc3891OCR.pdf      dan   
4    ../../Scans/Doc3713OCR.pdf      dan   
..                          ...      ...   
984  ../../Scans/Doc2908OCR.pdf      ita   
985  ../../Scans/Doc2907OCR.pdf      deu   
986  ../../Scans/Doc2906OCR.pdf      deu   
987  ../../Scans/Doc2905OCR.pdf      deu   
988  ../../Scans/Doc2904OCR.pdf      dan   

                                                  text  
0    SV 20 DB\n' Tørmessen, S(øren) Juell\nNorsk gr...  
1    SV 20 DB\nTønnessen,. SCøren) Juell.\n\nLærebo...  
2    | Hi. 10 . - oe -DB\n.  M@ytler, (AléXatide? F...  
3    Hi Je "DB\n(Tyskland)\nDen tyske Hvid-Bog se\n...  
4    | k\nPT 92 ' DB\nSverdrup, J(akob) R(osted).\n...  
..                                                 ...  
984  ST 35 DB\n| Oettingen, Alexander von\nDe pecca...  
985  82.20. on 

In [21]:
df_results.to_excel('scans_test.xlsx')